In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers.merge import concatenate
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2 
import numpy as np
import keras
import tensorflow as tf
from keras import backend as K 
from keras.utils import np_utils
from keras.layers import Layer
import keras.backend as K
import math 
from tensorflow.keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,Dropout, Dense, Input, concatenate,GlobalAveragePooling2D, AveragePooling2D,Flatten
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [2]:
train = pd.read_csv("ISIC_2019_Training_GroundTruth (2).csv")    # reading the csv file
train.head() 

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
train = train.astype({"MEL":'int', "NV":'int', "BCC":'int', "AK":'int', "BKL":'int', "DF":'int', "VASC":'int', "SCC":'int'}) 

In [29]:
mel_img=train[train['MEL']==1]
mel_img.reset_index(inplace=True)
mel_img.drop(columns=['index','UNK'],inplace=True)
mel_img.drop(mel_img.index[240:],inplace=True)
mel_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0000002,1,0,0,0,0,0,0,0
1,ISIC_0000004,1,0,0,0,0,0,0,0
2,ISIC_0000013,1,0,0,0,0,0,0,0
3,ISIC_0000022_downsampled,1,0,0,0,0,0,0,0
4,ISIC_0000026_downsampled,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
235,ISIC_0011350,1,0,0,0,0,0,0,0
236,ISIC_0011356,1,0,0,0,0,0,0,0
237,ISIC_0011366,1,0,0,0,0,0,0,0
238,ISIC_0011374,1,0,0,0,0,0,0,0


In [30]:
nv_img=train[train['NV']==1]
nv_img.reset_index(inplace=True)
nv_img.drop(columns=['index','UNK'],inplace=True)
nv_img.drop(nv_img.index[240:],inplace=True)
nv_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0000000,0,1,0,0,0,0,0,0
1,ISIC_0000001,0,1,0,0,0,0,0,0
2,ISIC_0000003,0,1,0,0,0,0,0,0
3,ISIC_0000006,0,1,0,0,0,0,0,0
4,ISIC_0000007,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
235,ISIC_0000354,0,1,0,0,0,0,0,0
236,ISIC_0000355,0,1,0,0,0,0,0,0
237,ISIC_0000357,0,1,0,0,0,0,0,0
238,ISIC_0000358,0,1,0,0,0,0,0,0


In [31]:
bcc_img=train[train['BCC']==1]
bcc_img.reset_index(inplace=True)
bcc_img.drop(columns=['index','UNK'],inplace=True)
bcc_img.drop(bcc_img.index[240:],inplace=True)
bcc_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0024331,0,0,1,0,0,0,0,0
1,ISIC_0024332,0,0,1,0,0,0,0,0
2,ISIC_0024345,0,0,1,0,0,0,0,0
3,ISIC_0024360,0,0,1,0,0,0,0,0
4,ISIC_0024403,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
235,ISIC_0028807,0,0,1,0,0,0,0,0
236,ISIC_0028814,0,0,1,0,0,0,0,0
237,ISIC_0028815,0,0,1,0,0,0,0,0
238,ISIC_0028818,0,0,1,0,0,0,0,0


In [32]:
bkl_img=train[train['BKL']==1]
bkl_img.reset_index(inplace=True)
bkl_img.drop(columns=['index','UNK'],inplace=True)
bkl_img.drop(bkl_img.index[240:],inplace=True)
bkl_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0010491,0,0,0,0,1,0,0,0
1,ISIC_0012086_downsampled,0,0,0,0,1,0,0,0
2,ISIC_0012090_downsampled,0,0,0,0,1,0,0,0
3,ISIC_0012103_downsampled,0,0,0,0,1,0,0,0
4,ISIC_0012117_downsampled,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
235,ISIC_0014055_downsampled,0,0,0,0,1,0,0,0
236,ISIC_0014074_downsampled,0,0,0,0,1,0,0,0
237,ISIC_0014083_downsampled,0,0,0,0,1,0,0,0
238,ISIC_0014120_downsampled,0,0,0,0,1,0,0,0


In [33]:
vasc_img=train[train['VASC']==1]
vasc_img.reset_index(inplace=True)
vasc_img.drop(columns=['index','UNK'],inplace=True)
vasc_img.drop(vasc_img.index[240:],inplace=True)
vasc_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0024370,0,0,0,0,0,0,1,0
1,ISIC_0024375,0,0,0,0,0,0,1,0
2,ISIC_0024402,0,0,0,0,0,0,1,0
3,ISIC_0024475,0,0,0,0,0,0,1,0
4,ISIC_0024662,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
235,ISIC_0071267,0,0,0,0,0,0,1,0
236,ISIC_0071298,0,0,0,0,0,0,1,0
237,ISIC_0071337,0,0,0,0,0,0,1,0
238,ISIC_0071383,0,0,0,0,0,0,1,0


In [34]:
scc_img=train[train['SCC']==1]
scc_img.reset_index(inplace=True)
scc_img.drop(columns=['index','UNK'],inplace=True)
scc_img.drop(scc_img.index[240:],inplace=True)
scc_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0024329,0,0,0,0,0,0,0,1
1,ISIC_0024372,0,0,0,0,0,0,0,1
2,ISIC_0024418,0,0,0,0,0,0,0,1
3,ISIC_0024450,0,0,0,0,0,0,0,1
4,ISIC_0024463,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
235,ISIC_0055204,0,0,0,0,0,0,0,1
236,ISIC_0055218,0,0,0,0,0,0,0,1
237,ISIC_0055302,0,0,0,0,0,0,0,1
238,ISIC_0055312,0,0,0,0,0,0,0,1


In [35]:
df_img=train[train['DF']==1]
df_img.reset_index(inplace=True)
df_img.drop(columns=['index','UNK'],inplace=True)
df_img.drop(df_img.index[240:],inplace=True)
df_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0024318,0,0,0,0,0,1,0,0
1,ISIC_0024330,0,0,0,0,0,1,0,0
2,ISIC_0024386,0,0,0,0,0,1,0,0
3,ISIC_0024396,0,0,0,0,0,1,0,0
4,ISIC_0024553,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
234,ISIC_0072970,0,0,0,0,0,1,0,0
235,ISIC_0073112,0,0,0,0,0,1,0,0
236,ISIC_0073141,0,0,0,0,0,1,0,0
237,ISIC_0073189,0,0,0,0,0,1,0,0


In [36]:
ak_img=train[train['AK']==1]
ak_img.reset_index(inplace=True)
ak_img.drop(columns=['index','UNK'],inplace=True)
ak_img.drop(ak_img.index[240:],inplace=True)
ak_img

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC
0,ISIC_0024468,0,0,0,1,0,0,0,0
1,ISIC_0024470,0,0,0,1,0,0,0,0
2,ISIC_0024511,0,0,0,1,0,0,0,0
3,ISIC_0024646,0,0,0,1,0,0,0,0
4,ISIC_0024654,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
235,ISIC_0056582,0,0,0,1,0,0,0,0
236,ISIC_0056613,0,0,0,1,0,0,0,0
237,ISIC_0056630,0,0,0,1,0,0,0,0
238,ISIC_0056643,0,0,0,1,0,0,0,0


In [37]:
col_var=[df_img, ak_img, mel_img, nv_img, bcc_img, bkl_img, vasc_img, scc_img]
train_image = []
for u in col_var:
    for i in tqdm(range(u.shape[0])):
        img = image.load_img('C:/Users/Lenovo/Downloads/F1/'+u['image'][i]+'.jpg',target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
X = np.array(train_image)

  7%|█████▍                                                                           | 16/239 [00:00<00:05, 39.38it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Lenovo/Downloads/F1/ISIC_0025668.jpg'